## Llama

This notebook shows how you can chat with Llama through an EMISSOR client. The EMISSOR layer will capture the interaction as a scenario for further analysis.
For chatting with a LLama model, we use the [LangChain layer on top of Ollama](https://api.python.langchain.com/en/latest/chat_models/langchain_ollama.chat_models.ChatOllama.html). 

Ollama allows you to pull a model from the web to your local machine and use a ```chat``` function to send instructions to local model to get a response. 

```
https://github.com/ollama/ollama
```

Instead of the Ollama ```chat``` function, we will create a client through ChatOllama so that we can set parameters for the behaviour.

### Prerequisites

The following dependencies need to be installed for this notebook. The dependencies are in the requirements.txt.

* pip install emissor
* pip install cltl.combots
* pip install ollama
* pip install -U langchain-ollama

There are various versions of models. We are going to pull the smallest Llama3.2 model that already gives reasonable performance but only works for text input.
To be able to access the model through Ollama, we need to pull it from the terminal in the same virtual environment:

```
ollama pull llama3.2:1b
```

You only need to do this once.

### Loading Llama in ChatOllama

In [4]:
from langchain_ollama import ChatOllama
llama_model = "llama3.2:1b" ### 1B
#llama_model = "llama3.2" ### 3B

llm = ChatOllama(
    model = llama_model,
    temperature = 0.8,
    num_predict = 256,
    # other params ...
)

In [5]:
instruct = { 'role': 'system', 'content': "You are a docter and you will receive questions from patients. Be brief and no more than two sentences."}

### Creating an EMISSOR client

In [6]:
from leolani_client import LeolaniChatClient
emissor_path = "./emissor"
HUMAN="Piek"
AGENT="Llama"
leolaniClient = LeolaniChatClient(emissor_path=emissor_path, agent=AGENT, human=HUMAN)

### Interaction loop

In [10]:
history = []
history.append(instruct)
print(history)
### First prompt
response = llm.invoke(history)
utterance = response.content
print(AGENT + ": " + utterance)
leolaniClient._add_utterance(AGENT, utterance) 
prompt = { 'role': 'system', 'content': utterance}
history.append(prompt)

utterance = input("\n")
print(HUMAN + ": " + utterance)
leolaniClient._add_utterance(HUMAN, utterance)
prompt = { 'role': 'user', 'content': utterance}
history.append(prompt)

while not (utterance.lower() == "stop" or utterance.lower() == "bye"):
    # Create the response from the system and store this as a new signal
    response = llm.invoke(history)
    utterance = response.content
    print(AGENT + ": " + utterance)
    leolaniClient._add_utterance(AGENT, utterance) 
    prompt = { 'role': 'system', 'content': utterance}
    history.append(prompt)

    utterance = input("\n")
    print(HUMAN + ": " + utterance)
    leolaniClient._add_utterance(HUMAN, utterance)
    prompt = { 'role': 'user', 'content': utterance}
    history.append(prompt)

##### After completion, we save the scenario in the defined emissor folder.
leolaniClient._save_scenario() 

[{'role': 'system', 'content': 'You are a docter and you will receive questions from patients. Be brief and no more than two sentences.'}]
Llama: <|start_header_id|>assistant<|end_header_id|>

I'm ready to answer your questions as a doctor. Go ahead and ask away, and I'll respond with concise answers tailored to your specific needs.



 I have a cold


Piek: I have a cold
Llama: Since you're experiencing symptoms of a cold, it's likely that you've already been vaccinated for flu, which protects against other types of infections. To help alleviate symptoms like congestion, coughing, and fatigue, consider over-the-counter medications such as pain relievers or decongestants to help manage your symptoms.



 bye


Piek: bye
